## Assignment: finding the best ANN layer for different brain regions

1. Pick ~5 layers in your model (try picking the mid/late layers since they yield smaller, features)
2. Collect their features
3. For each region of interest:
    3.1 train a ridge regression model
    3.2 evaluate it on a held-out subject
    3.3 keep a log of the correlation score somewhere
4. Find the layer with the highest correlation score for each region

Ideally, make a bar plot with your results where X = layer name and Y = correlation score. One plot for each region of interest.

In [2]:
from neuroai.datasets import NSDAllSubjectSingleRegion
from neuroai.utils import ForwardHook
import torch.nn as nn
import torch
from neuroai.utils.regression import RidgeModel, ridge_regression

## here's a bunch of functions from the other notebook to make your life easier
def collect_features_and_voxels(
    dataset: NSDAllSubjectSingleRegion, 
    backbone_model: nn.Module, 
    hook: ForwardHook, 
    device: str, 
    subject_id: str
):
    
    ## this is where we dump our data
    all_features = []
    all_fmri_voxels = []

    with torch.no_grad():
        for i in range(len(dataset)):
            image: torch.Tensor = dataset[i]["image"]

            ## (voxels) -> (1, voxels)
            fmri_data: torch.Tensor = dataset[i]["fmri_response"][subject_id].unsqueeze(0)

            ## (channels, height, width) -> (1, channels, height, width)
            image = image.unsqueeze(0)
            image = image.to(device)
            y = backbone_model(image)

            ## making sure that we're moving stuff back to the RAM with .cpu()
            hook_output = hook.output.cpu()
            all_features.append(hook_output)
            all_fmri_voxels.append(fmri_data.cpu())

    return {
        "features": torch.cat(all_features, dim=0),
        "voxels": torch.cat(all_fmri_voxels, dim=0)
    }

In [ ]:
from neuroai.models import CLIPRN50

backbone_model = CLIPRN50(pretrained=True, download_root="./pretrained_checkpoints")
backbone_model.eval()
device = "cuda" if torch.cuda.is_available() else "cpu"
backbone_model = backbone_model.to(device)

## first, look at the model and select a few layers
print(backbone_model)

In [4]:
region = "FFA"

layers_to_evaluate = [
    ## select a few layers to evaluate over
    "model.layer4.0.conv1"
]

In [ ]:
dataset = NSDAllSubjectSingleRegion(
    folder="./datasets/nsd",
    region=region,
    transforms=backbone_model.transforms,
    subset="train",
    train_test_split=0.8
)

features_from_different_layers = {}

for layer_name in layers_to_evaluate:
    print(f"Processing layer {layer_name}")

    hook = ForwardHook(
        model=backbone_model,
        hook_layer_name=layer_name
    )

    collected_data = collect_features_and_voxels(
        dataset=dataset,
        backbone_model=backbone_model,
        hook=hook,
        device=device,
        subject_id="s1"
    )

    features = collected_data["features"]
    voxels = collected_data["voxels"]

    print(f"Features shape: {features.shape}, Voxels shape: {voxels.shape}")

    features_from_different_layers[layer_name] = features
    hook.close()  # clean up the hook